In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, recall_score, precision_score, det_curve
from evaluate_metrics import compute_eer, compute_cllr, compute_min_dcf, calculate_minDCF_EER_CLLR_actDCF
import librosa
from IPython.display import Audio, display
import librosa.display
import matplotlib.pyplot as plt
import random
def recursive_list_files(directory, extension=None):
    """
    Recursively list files in a directory
    :param directory: directory to list files from
    :param extension: file extension to filter by
    :return: list of files in directory
    """
    files = []
    for root, dirs, filenames in os.walk(directory, followlinks=True):
        for filename in filenames:
            if extension is None or filename.endswith(extension):
                files.append(os.path.join(root, filename))
    return files

def play_and_show(file_path):
    """
    Play and show an audio file
    :param file_path: path to audio file
    """
    y, sr = librosa.load(file_path, sr=None)
    fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True)
    display(Audio(file_path, rate=16000))
    # Compute the spectrogram
    S = np.abs(librosa.stft(y, n_fft=2048, hop_length=240, win_length=480, window='hamming'))
    D = librosa.amplitude_to_db(S, ref=np.max)
    img = librosa.display.specshow(D, y_axis='linear', x_axis='time', n_fft=2048, hop_length=240, win_length=480, 
                               sr=sr, ax=ax)
    plt.show()

    

# ASVspoof 2019

In [3]:
la19_df = pd.read_csv('/datab/Dataset/ASVspoof/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt', sep=' ', header=None)
la19_df.columns = ['spid', 'utt', 'phy', 'att', 'label']
la19_df

,spid,utt,phy,att,label
0,LA_0079,LA_T_1138215,-,-,bonafide
1,LA_0079,LA_T_1271820,-,-,bonafide
2,LA_0079,LA_T_1272637,-,-,bonafide
3,LA_0079,LA_T_1276960,-,-,bonafide
4,LA_0079,LA_T_1341447,-,-,bonafide
...,...,...,...,...,...
25375,LA_0098,LA_T_9717580,-,A06,spoof
25376,LA_0098,LA_T_9779814,-,A06,spoof
25377,LA_0098,LA_T_9783312,-,A06,spoof
25378,LA_0098,LA_T_9839348,-,A06,spoof


In [7]:
# la19_df['spid'].value_counts()
la19_bona_df = la19_df[la19_df['label']=='bonafide']
# import time
# start = time.time()
for idx, row in tqdm(la19_bona_df.iterrows(), total=la19_bona_df.shape[0]):
    anchor = row['utt']
    other_real = la19_df[(la19_df['label']=='bonafide') & (la19_df['utt']!=anchor)].sample(1)['utt'].values[0]
    other_spid_real = la19_df[(la19_df['label']=='bonafide') & (la19_df['spid']!=row['spid'])].sample(1)['utt'].values[0]
    same_spid_spoof = la19_df[(la19_df['label']=='spoof') & (la19_df['spid']==row['spid'])].sample(1)['utt'].values[0]
    diff_spid_spoof = la19_df[(la19_df['label']=='spoof') & (la19_df['spid']!=row['spid'])].sample(1)['utt'].values[0]

# print('Time taken:', time.time()-start)


  0%|          | 0/2580 [00:00<?, ?it/s]

100%|██████████| 2580/2580 [00:00<00:00, 17378.48it/s]


In [11]:
# group by spid
la19_bona_df_grouped = la19_bona_df.groupby('spid')['utt'].apply(list).reset_index()
la19_bona_df_grouped
# convert to dictionary
la19_bona_dict = la19_bona_df_grouped.set_index('spid').T.to_dict('list')
la19_bona_dict
# save to object file
import pickle
with open('DATA/SASV2022/scp/speaker_bona_train.pkl', 'wb') as f:
    pickle.dump(la19_bona_dict, f)

In [14]:
la19_spoof_df = la19_df[la19_df['label']=='spoof']
# group by spid
la19_spoof_df_grouped = la19_spoof_df.groupby('spid')['utt'].apply(list).reset_index()
la19_spoof_df_grouped
# convert to dictionary
la19_spoof_dict = la19_spoof_df_grouped.set_index('spid').T.to_dict('list')
la19_spoof_dict
# save to object file
import pickle
with open('DATA/SASV2022/scp/speaker_spoof_train.pkl', 'wb') as f:
    pickle.dump(la19_spoof_dict, f)




In [ ]:
# load speaker dictionary
with open('DATA/SASV2022/scp/speaker_bona_train.pkl', 'rb') as f:
    speaker_bona_dict = pickle.load(f)

In [24]:
speaker_bona = pickle.load(open('DATA/SASV2022/scp/speaker_bona_train.pkl', 'rb'))
list_IDs = []
# extract all the utt from the dictionary
for key, value in speaker_bona.items():
    for i in value:
        list_IDs+=i

# shuffle the list
random.shuffle(list_IDs)